<h1> QUESTION 1: Postal Code Data Frame from WIKI Page </h1>

In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import lxml
import numpy as np

In [2]:
# I Struggled with beautfulsoup packageing in getting the rows logged into the DF,
# so i found a much simplier way using Pandas

url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
table = pd.read_html(url)[0]
table.head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [3]:
#Start of the 1st clean through removing 'not assinged'

table = table.astype('str')
table2 = table[table.Borough != "Not assigned"]
table2.head()

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [4]:
#Check for Null values in Neighbourhood

table2['Neighborhood'].isnull().values.any()

False

!!! with no Null values in neighjbourhood its not neccesary to code out the nulls for the Borough name

In [5]:
table2.shape

(103, 3)

<h1> Question 2:  Lat and Long for Neighborhoods </h2>

In [6]:
#import Long and Lat CSV
Coords = pd.read_csv('https://cocl.us/Geospatial_data')

In [7]:
#merge the 2 tables
newtable = pd.merge(table2, Coords, on='Postal Code', how='right')
newtable.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


<h1> Question 3: Clusting of Codes </h1>

In [192]:
import json # library to handle JSON files

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Libraries imported.


In [9]:
!pip -q install folium
import folium 


In [10]:
map_Toronto = folium.Map(location=['43.6532', '-79.3832'], zoom_start=10)


for lat, lng, borough, neighborhood in zip(newtable['Latitude'], newtable['Longitude'], newtable['Borough'], newtable['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)  

map_Toronto